In [1]:
import urllib
import json
import re
from ipywidgets import interact, interactive
import ipywidgets as widgets
from IPython.core.display import display, HTML

In [2]:
def getExoMastTCEsForTIC(tessId):
    url = f'https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/{tessId}/tces/'
    response = urllib.request.urlopen(url)
    data = response.read()      # a `bytes` object
    text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
    respObj = json.loads(text)
    return respObj['TCE']

def getExoMastUrlsForTIC(tessId):
    tces = getExoMastTCEsForTIC(tessId)
    tces.sort()
    urlPrefix = 'https://exo.mast.stsci.edu/exomast_planet.html'
    return list(map(lambda x: (x, f"{urlPrefix}?planet=TIC{tessId}" + re.sub(r"[-:_]", "", x.upper()))
                    , tces))

def showExoMastUrlsForTIC(tessId):
    res = getExoMastUrlsForTIC(tessId)
    
    display(HTML(f'<h3>TCEs for {tessId}</h3>'))
    
    if len(res) < 1:
        display(HTML('&nbsp;&nbsp; none found.'))
        return 0
    # normal case
    out = '<ul>'
    for aRes in res:
        out += f'\n  <li><a href="{aRes[1]}" target="_exomast">{aRes[0]}</a>'
    out += '\n</ul>'
    display(HTML(out))    
    display(HTML('<p><em>Tips:</em> On ExoMAST, <code>Related Links > Vetting Summary Page</code> (pdf) is a useful summary.</p>'))
    return len(res)

def showExoFOPUrlsForTIC(tessId):
    out = f'''<hr>
    <div>
      <p><a href="https://exofop.ipac.caltech.edu/tess/target.php?id={tessId}" target="_exofop">Status on ExoFOP</a> - if there are follow-ups of the target</p>
      <br><em>Tips:</em>Check the follow up status to see if the target has potential exoplanets:
      <ul>
        <li>TESS Disposition ,
        <li>TFOPWG Disposition
      </ul>
      Example: 
      <ul>
        <li>TESS = <code>PC</code>: Planet Candidate 
        <li>TFOPWG = <code>CP</code>: Confirmed Planet
      </ul>
    </div>'''
    display(HTML(out))
    return

def showLinksForMoreInfo(tic):
    tic = re.sub(r'^tic\s*', '', tic, flags=re.IGNORECASE)
    display('')
    showExoMastUrlsForTIC(tic)
    showExoFOPUrlsForTIC(tic)
    display(HTML('<hr>More tips on <a href="https://www.zooniverse.org/projects/nora-dot-eisner/planet-hunters-tess/talk/2108/812359" target="_talk">Planet Hunters TESS Talk</a>'))
    return



In [3]:
# shwo a widget to let user enters an ID
interactive(showLinksForMoreInfo, tic = widgets.Text(description="TIC id", value ="388104525"))


interactive(children=(Text(value='388104525', description='TIC id'), Output()), _dom_classes=('widget-interact…